<a href="https://colab.research.google.com/github/pvuppal/prudhvi/blob/main/IT7103_prudhvivuppalapati_assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('real_estate_value.csv')

In [ ]:
df.head()

,HouseAge,DistanceToMRT,NoOfStores,Latitude,Longitude,UnitPrice
0,32.0,84.87882,10,24.98298,121.54024,37.9
1,19.5,306.59470,9,24.98034,121.53951,42.2
2,13.3,561.98450,5,24.98746,121.54391,47.3
3,13.3,561.98450,5,24.98746,121.54391,54.8
4,5.0,390.56840,5,24.97937,121.54245,43.1


In [ ]:
df.dtypes

HouseAge         float64
DistanceToMRT    float64
NoOfStores         int64
Latitude         float64
Longitude        float64
UnitPrice        float64
dtype: object

In [ ]:
df = df.dropna()

In [ ]:
y = df['UnitPrice']
x = df.drop('UnitPrice', axis=1)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
col1 = []
col2 = []

for column in x.columns:
    if df[column].skew() > -1 and df[column].skew() < 1:
        col1.append(column)
    else:
        col2.append(column)

In [ ]:
symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, col1),
    ('numeric_skewed', skewed_pipeline, col2),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [ ]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [ ]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(train_transformed, train_y)
pred_dt = decision_tree.predict(test_transformed)

In [ ]:
mse_dt = mean_squared_error(test_y, pred_dt)
r2_dt = r2_score(test_y, pred_dt)

In [ ]:
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(train_transformed, train_y)
pred_rf = random_forest.predict(test_transformed)

In [ ]:
mse_rf = mean_squared_error(test_y, pred_rf)
r2_rf = r2_score(test_y, pred_rf)

In [ ]:
print("Mean Squared Error (Decision Tree):", mse_dt)
print("R-squared (Decision Tree):", r2_dt)
print("Mean Squared Error (Random Forest):", mse_rf)
print("R-squared (Random Forest):", r2_rf)

Mean Squared Error (Decision Tree): 55.57156626506025
R-squared (Decision Tree): 0.6687431396370993
Mean Squared Error (Random Forest): 34.83787655745048
R-squared (Random Forest): 0.7923347066539833
